# Tomo 5B

In [1]:
import os
from collections import Counter, defaultdict
from pprint import pprint
from string import punctuation

import nltk
import numpy as np
from gensim import corpora, models
from nltk.corpus import stopwords
from scipy import spatial

## Loading data

In [2]:
def key_sort_files(x):
    return int(x[:-4].split('-')[0])

path = 'data/aux/biblioteca/text_parts/5b/'
unsorted_file_list = [filename for filename in os.listdir(path) if filename.endswith('.txt')]
file_list = sorted(unsorted_file_list, key=key_sort_files)

raw_texts = []
for filename in file_list:
    with open(path + filename) as f:
        raw_texts.append(f.read())

## Cleaning data

In [3]:
chars = []
for text in raw_texts:
    for c in text:
        if not c.isalnum():
            chars.append(c)

In [4]:
characters = set(chars)

In [5]:
with open('data/stopwords/spanish_stopwords.txt') as f:
    sp_stopwords = list(set(map(str.strip, f.readlines())))

with open('data/stopwords/my_stopwords.txt') as f:
    my_stopwords = list(set(map(str.strip, f.readlines())))

stop = stopwords.words('spanish') + sp_stopwords + my_stopwords + list(punctuation) + list(characters)

In [6]:
def clean(s):
    r = s.lower().strip()
    for c in characters:
        r = r.replace(c, ' ')
    r = r.replace('farc ep', 'farc-ep')
    r = r.replace('confianz a', 'confianza')
    r = r.replace('cons trucción', 'construcción')
    rs = [w for w in nltk.word_tokenize(r) if w not in stop and len(w) > 2 and not w.isdecimal()]
    r = ' '.join(rs)
    return r

In [7]:
cleaned_texts = [clean(text) for text in raw_texts]

In [8]:
char_counter = defaultdict(int)
for c in characters:
    for text in cleaned_texts:
        if c in text:
            char_counter[c]+=text.count(c)
char_counter

defaultdict(int, {' ': 62162, '-': 515})

## Preparing data

In [9]:
documents = cleaned_texts

In [10]:
document = ' '.join(documents)

In [11]:
# Only texts that contain MORE THAN or EQUALS TO 100 words
texts = [[word for word in document.split()] for document in documents if len(document.split()) >= 100]

In [12]:
len(texts)

85

In [13]:
dictionary = corpora.Dictionary(texts)

In [14]:
len(dictionary)

9474

## Processing data

In [15]:
%%time

# lsi_topics_lists = []
lda_topics_lists = []
hdp_topics_lists = []

MIN_DFs = [1, 2, 4, 7]
MAX_DFs = [0.6, 0.7, 0.8, 0.9]
MAX_FTs = [100, 1000, len(dictionary)]
N_TOPICSs = [10, len(texts), 100]

total_iterations = len(MIN_DFs) * len(MAX_DFs) * len(MAX_FTs) * len(N_TOPICSs)
iteration = 1
for MIN_DF in MIN_DFs:
    for MAX_DF in MAX_DFs:
        for MAX_FT in MAX_FTs:
            dictionary = corpora.Dictionary(texts)
            dictionary.filter_extremes(no_below=MIN_DF, no_above=MAX_DF, keep_n=MAX_FT)
            corpus = [dictionary.doc2bow(text) for text in texts]
            tfidf = models.TfidfModel(corpus)
            corpus_tfidf = tfidf[corpus]
            
            for N_TOPICS in N_TOPICSs:
                print('Iteration: {} / {}'.format(iteration, total_iterations), \
                      '- MIN_DF:', MIN_DF, '- MAX_DF:', MAX_DF, '- MAX_FT:', MAX_FT, '- N_TOPICS:', N_TOPICS)
                iteration += 1
                
                # LSI
#                 lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=N_TOPICS)
#                 corpus_lsi = lsi[corpus_tfidf]
#                 ts = lsi.show_topics(lsi.num_topics, formatted=False)
#                 result = (ts, (MIN_DF, MAX_DF, MAX_FT, N_TOPICS), corpus_lsi)
#                 lsi_topics_lists.append(result)
            
                # LDA
                lda = models.LdaModel(corpus, id2word=dictionary, num_topics=N_TOPICS, random_state=0)
                corpus_lda = lda[corpus]
                ts = sorted(lda.show_topics(lda.num_topics, formatted=False))
                result = (ts, (MIN_DF, MAX_DF, MAX_FT, N_TOPICS), corpus_lda)
                lda_topics_lists.append(result)
            
                # HDP
                hdp = models.HdpModel(corpus, id2word=dictionary, random_state=0)
                corpus_hdp = hdp[corpus]
                ts = hdp.show_topics(len(hdp.show_topics(num_topics=-1)), num_words=10, formatted=False)
                result = (ts, (MIN_DF, MAX_DF, MAX_FT, N_TOPICS), corpus_hdp)
                hdp_topics_lists.append(result)

Iteration: 1 / 144 - MIN_DF: 1 - MAX_DF: 0.6 - MAX_FT: 100 - N_TOPICS: 10
Iteration: 2 / 144 - MIN_DF: 1 - MAX_DF: 0.6 - MAX_FT: 100 - N_TOPICS: 85
Iteration: 3 / 144 - MIN_DF: 1 - MAX_DF: 0.6 - MAX_FT: 100 - N_TOPICS: 100
Iteration: 4 / 144 - MIN_DF: 1 - MAX_DF: 0.6 - MAX_FT: 1000 - N_TOPICS: 10
Iteration: 5 / 144 - MIN_DF: 1 - MAX_DF: 0.6 - MAX_FT: 1000 - N_TOPICS: 85
Iteration: 6 / 144 - MIN_DF: 1 - MAX_DF: 0.6 - MAX_FT: 1000 - N_TOPICS: 100
Iteration: 7 / 144 - MIN_DF: 1 - MAX_DF: 0.6 - MAX_FT: 9474 - N_TOPICS: 10
Iteration: 8 / 144 - MIN_DF: 1 - MAX_DF: 0.6 - MAX_FT: 9474 - N_TOPICS: 85
Iteration: 9 / 144 - MIN_DF: 1 - MAX_DF: 0.6 - MAX_FT: 9474 - N_TOPICS: 100
Iteration: 10 / 144 - MIN_DF: 1 - MAX_DF: 0.7 - MAX_FT: 100 - N_TOPICS: 10
Iteration: 11 / 144 - MIN_DF: 1 - MAX_DF: 0.7 - MAX_FT: 100 - N_TOPICS: 85
Iteration: 12 / 144 - MIN_DF: 1 - MAX_DF: 0.7 - MAX_FT: 100 - N_TOPICS: 100
Iteration: 13 / 144 - MIN_DF: 1 - MAX_DF: 0.7 - MAX_FT: 1000 - N_TOPICS: 10
Iteration: 14 / 144 - M

Iteration: 109 / 144 - MIN_DF: 7 - MAX_DF: 0.6 - MAX_FT: 100 - N_TOPICS: 10
Iteration: 110 / 144 - MIN_DF: 7 - MAX_DF: 0.6 - MAX_FT: 100 - N_TOPICS: 85
Iteration: 111 / 144 - MIN_DF: 7 - MAX_DF: 0.6 - MAX_FT: 100 - N_TOPICS: 100
Iteration: 112 / 144 - MIN_DF: 7 - MAX_DF: 0.6 - MAX_FT: 1000 - N_TOPICS: 10
Iteration: 113 / 144 - MIN_DF: 7 - MAX_DF: 0.6 - MAX_FT: 1000 - N_TOPICS: 85
Iteration: 114 / 144 - MIN_DF: 7 - MAX_DF: 0.6 - MAX_FT: 1000 - N_TOPICS: 100
Iteration: 115 / 144 - MIN_DF: 7 - MAX_DF: 0.6 - MAX_FT: 9474 - N_TOPICS: 10
Iteration: 116 / 144 - MIN_DF: 7 - MAX_DF: 0.6 - MAX_FT: 9474 - N_TOPICS: 85
Iteration: 117 / 144 - MIN_DF: 7 - MAX_DF: 0.6 - MAX_FT: 9474 - N_TOPICS: 100
Iteration: 118 / 144 - MIN_DF: 7 - MAX_DF: 0.7 - MAX_FT: 100 - N_TOPICS: 10
Iteration: 119 / 144 - MIN_DF: 7 - MAX_DF: 0.7 - MAX_FT: 100 - N_TOPICS: 85
Iteration: 120 / 144 - MIN_DF: 7 - MAX_DF: 0.7 - MAX_FT: 100 - N_TOPICS: 100
Iteration: 121 / 144 - MIN_DF: 7 - MAX_DF: 0.7 - MAX_FT: 1000 - N_TOPICS: 10
I

In [16]:
def extract_vocabulary(ts):
    vocab = set()
    for t in ts:
        words = [word for word, weight in t]
        vocab = vocab.union(words)
    vocab = sorted(vocab)
    r = {w: i for i, w in enumerate(vocab)}
    return r

def get_weights(t):
    return dict(t)

def create_vector(vocab, weights):
    r = [0] * len(vocab)
    for wo in weights:
        r[vocab[wo]] = weights[wo]
    return r

def vectorize(ts):
    r = []
    ts = [t for i, j, t in ts]
    vocab = extract_vocabulary(ts)
    for t in ts:
        weights = get_weights(t)
        vector = create_vector(vocab, weights)
        r.append(vector)
    return r

In [17]:
def two_topics_distance(t1, t2):
    r = spatial.distance.cosine(t1, t2)
    return r

def multiple_topics_distance(ts):
    ds = []
    for i, t1 in enumerate(ts[:-1]):
        for j in range(i + 1, len(ts)):
            t2 = ts[j]
            d = two_topics_distance(t1, t2)
            ds.append(d)
    return np.mean(ds)

def top_n_used(ts_list, n):
    total_weights = defaultdict(float)
    corpus = ts_list[2]
    for doc in corpus:
        for topic, weight in doc:
            total_weights[topic] += abs(weight)
    total_weights = dict(total_weights)
    counter = Counter(total_weights)
    topics_dct = dict(ts_list[0])
    r = ([(weight, topic_id, topics_dct[topic_id])
             for topic_id, weight in counter.most_common(n)], ts_list[1], corpus)
    return r

def best_topics(ts_lists, n=10):
    r = []
    for ts_list in ts_lists:
        top_n_ts_list = top_n_used(ts_list, n)
        ts_vector = vectorize(top_n_ts_list[0])
        d = multiple_topics_distance(ts_vector)
        r.append((d, top_n_ts_list))
    r = sorted(r, reverse=True)
    return r[0]

In [18]:
%%time

# print('LSI')
# lsi_best_topics_list = best_topics(lsi_topics_lists)
print('LDA')
lda_best_topics_list = best_topics(lda_topics_lists)
print('HDP')
hdp_best_topics_list = best_topics(hdp_topics_lists)

LDA
HDP
CPU times: user 2min 55s, sys: 2min 29s, total: 5min 25s
Wall time: 1min 27s


## Results

### LSI

In [19]:
# lsi_best_topics_list[1][0]

### LDA

In [20]:
lda_best_topics_list[1][0]

[(3.933880852535367,
  24,
  [('colombianos', 0.066531695),
   ('confianza', 0.045571104),
   ('medidas', 0.030211462),
   ('búsqueda', 0.029236639),
   ('construcción', 0.028218633),
   ('personas', 0.027958073),
   ('armado', 0.027270539),
   ('país', 0.023853913),
   ('fuerzas', 0.021859759),
   ('cese', 0.021099994)]),
 (3.7093954533338547,
  37,
  [('fuego', 0.060862582),
   ('cese', 0.056697749),
   ('años', 0.044493277),
   ('país', 0.02776679),
   ('bilateral', 0.025121357),
   ('violencia', 0.024738669),
   ('colombiano', 0.02262843),
   ('personas', 0.020685486),
   ('medidas', 0.019310148),
   ('república', 0.019141063)]),
 (3.5121290720999241,
  97,
  [('colombianos', 0.063293897),
   ('país', 0.033196405),
   ('cese', 0.029324625),
   ('vida', 0.028835677),
   ('años', 0.027550448),
   ('fuego', 0.027180497),
   ('hoy', 0.026900103),
   ('calle', 0.026135329),
   ('jefe', 0.024751348),
   ('humberto', 0.023710962)]),
 (3.4431252107024193,
  91,
  [('reparación', 0.08787019

### HDP

In [21]:
hdp_best_topics_list[1][0]

[(10.004880107903761,
  0,
  [('comisión', 0.009658933618318875),
   ('reparación', 0.0079674155372349929),
   ('humanos', 0.0072762462549024586),
   ('especial', 0.0063476734787808091),
   ('repetición', 0.0059103347632380593),
   ('medidas', 0.0056243646270251859),
   ('armado', 0.0055727018667878631),
   ('integral', 0.0051891020598585766),
   ('sistema', 0.0051625145769254241),
   ('responsabilidad', 0.0051207242428069613)]),
 (6.6148572178738627,
  9,
  [('hoy', 0.0063624645039302352),
   ('colombianos', 0.0058015116382197467),
   ('nunca', 0.0054919733128186866),
   ('fuego', 0.0052225921729367993),
   ('confianza', 0.004291832497593487),
   ('miembros', 0.0041643038902325156),
   ('firma', 0.0041638583431859201),
   ('derecho', 0.0041501738277199985),
   ('conductas', 0.0040668274560378814),
   ('artefactos', 0.0040248576885362946)]),
 (5.837210701654139,
  1,
  [('reparación', 0.016085299618625263),
   ('especial', 0.0061698268732447445),
   ('delitos', 0.0054472570812826573),
